# ENTRENAMIENTO FINAL SENTINEL HUD (YOLOv5)

**Asignatura**: Vision Artificial Avanzada  
**Proyecto**: Sentinel HUD - Sistema de Monitorizacion Industrial  
**Autor**: Diego Sanchez

---

## 1. Configuracion del Entorno
Clonamos el repositorio oficial de YOLOv5 e instalamos las dependencias necesarias.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

import torch
import os
import glob
from IPython.display import Image, clear_output, display

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

## 2. Preparacion del Dataset
Subir el archivo `dataset_final.zip` generado localmente a la raiz de Colab antes de ejecutar esta celda.

In [ ]:
if os.path.exists('../dataset_final.zip'):
    !unzip -q -o ../dataset_final.zip -d ../dataset_final
    print("[OK] Dataset descomprimido correctamente.")
else:
    print("[INFO] Usando carpeta '../../dataset_final' existente.")

## 3. Entrenamiento (Transfer Learning)
Entrenamos **YOLOv5s** (version small) por su equilibrio entre velocidad (inference time) y precision (mAP).

In [ ]:
data_path = '../dataset_final/custom_data.yaml'
if not os.path.exists(data_path):
    data_path = 'custom_data.yaml'
    
!python train.py --img 640 --batch 16 --epochs 100 --data {data_path} --weights yolov5s.pt --name sentinel_exp --cache

## 4. Analisis de Resultados Automatico

In [ ]:
folders = glob.glob('runs/train/sentinel_exp*')
if folders:
    latest_folder = max(folders, key=os.path.getctime)
    print(f"[INFO] Cargando evidencias desde: {latest_folder}")
    
    for file in ['results.png', 'confusion_matrix.png', 'F1_curve.png']:
        img_path = os.path.join(latest_folder, file)
        if os.path.exists(img_path):
            print(f"[OK] Mostrando {file}")
            display(Image(filename=img_path, width=800))
        else:
            print(f"[ERR] {file} no generado aun.")
else:
    print("[ERR] No se encontro ninguna carpeta de entrenamiento 'sentinel_exp'.")

## 5. Exportacion del Modelo

In [ ]:
folders = glob.glob('runs/train/sentinel_exp*')
if folders:
    latest_folder = max(folders, key=os.path.getctime)
    best_model = os.path.join(latest_folder, 'weights/best.pt')
    print(f"[OK] Tu mejor modelo esta en: {os.path.abspath(best_model)}")
    
    try:
        from google.colab import files
        files.download(best_model)
    except:
        pass